In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os.path as osp
import os
import subprocess
import datetime
from tqdm import tqdm
from utils.basics import PRCS_DATA_DIR, ATLASES_DIR, ATLAS_NAME, PRJ_DIR, CODE_DIR
from sfim_lib.io.afni import load_netcc
from sfim_lib.plotting.fc_matrices import hvplot_fc

/data/SFIMJGC_HCP7T/Apps/envs/bold_waves_2024a/lib/python3.9/site-packages/nxviz/__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.4 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.4, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


In [2]:
import getpass
username = getpass.getuser()
print(username)

javiergc


In [3]:
import os
port_tunnel = int(os.environ['PORT2'])
print('++ INFO: Second Port available: %d' % port_tunnel)

++ INFO: Second Port available: 34257


In [4]:
ATLAS_NAME = 'Power264'

In [5]:
ATLAS_DIR = osp.join(ATLASES_DIR,ATLAS_NAME)

# 1. Load Dataset Information

In [6]:
dataset_info_df = pd.read_csv(osp.join(PRJ_DIR,'resources','good_scans.txt'))
dataset_info_df = dataset_info_df.set_index(['Subject','Session'])
print('++ Number of scans: %s scans' % dataset_info_df.shape[0])

++ Number of scans: 436 scans


# 2. Create Swarm Script to Extract ROI TS from fully denoised data

In [7]:
script_path = osp.join(PRJ_DIR,f'swarm.{username}',f'N03b_extract_ROI_ts.{ATLAS_NAME}.OC.sh')
print(script_path)

/data/SFIMJGC_HCP7T/BCBL2024/swarm.javiergc/N03b_extract_ROI_ts.Power264.OC.sh


In [8]:
log_path = osp.join(PRJ_DIR,f'logs.{username}',f'N03b_extract_ROI_ts.{ATLAS_NAME}.OC.log')
if not osp.exists(log_path):
    os.makedirs(log_path)
print(log_path)

/data/SFIMJGC_HCP7T/BCBL2024/logs.javiergc/N03b_extract_ROI_ts.Power264.OC.log


In [10]:
with open(script_path, 'w') as the_file:
    the_file.write('# Script Creation Date: %s\n' % str(datetime.date.today()))
    the_file.write(f'# swarm -f {script_path} -g 16 -t 8 -b 10 --time 00:20:00 --logdir {log_path} --partition quick,norm --module afni\n')
    the_file.write('\n')
    for sbj,ses in list(dataset_info_df.index):
        atlas_path  = f'{ATLASES_DIR}/{ATLAS_NAME}/{ATLAS_NAME}.nii.gz' 
        inset_path  = f'pb04.{sbj}.r01.combine+tlrc'
        outset_path = f'{sbj}.r01.OC.{ATLAS_NAME}'
        the_file.write(f'cd {PRCS_DATA_DIR}/{sbj}/D02_Preproc_fMRI_{ses}; 3dNetCorr -overwrite -automask_off -in_rois {atlas_path} -inset {inset_path} -prefix {outset_path}; 3dROIstats -quiet -mask {atlas_path} {inset_path} > pb03.{sbj}.r01.OC.{ATLAS_NAME}_000.netts \n')
the_file.close()     

```bash
cd /data/SFIMJGC_HCP7T/BCBL2024/swarm.javiergc
swarm -f /data/SFIMJGC_HCP7T/BCBL2024/swarm.javiergc/N03b_extract_ROI_ts.Power264.OC.sh -g 16 -t 8 -b 10 --time 00:20:00 --logdir /data/SFIMJGC_HCP7T/BCBL2024/logs.javiergc/N03b_extract_ROI_ts.Power264.OC.log --partition quick,norm --module afni

# 3. Check all expected datasets were processed

In [12]:
for sbj,ses in list(dataset_info_df.index):
    netcc_path = osp.join(PRCS_DATA_DIR,sbj,f'D02_Preproc_fMRI_{ses}',f'{sbj}.r01.OC.{ATLAS_NAME}_000.netcc')
    if not osp.exists(netcc_path):
        print('++ WARNING: %s is missing'% netcc_path)

In [11]:
import xarray as xr
import panel as pn

In [12]:
roi_info_path = osp.join(ATLAS_DIR,f'{ATLAS_NAME}.roi_info.csv')
roi_info_df   = pd.read_csv(roi_info_path)

In [13]:
fcs = xr.DataArray(dims=['scan','roi_x','roi_y'], coords={'scan':['.'.join([sbj,ses]) for sbj,ses in dataset_info_df.index],'roi_x':list(roi_info_df['ROI_Name']),'roi_y':list(roi_info_df['ROI_Name'])})

In [14]:
for sbj,ses in tqdm(list(dataset_info_df.index)):
    netcc_path = osp.join(PRCS_DATA_DIR,sbj,f'D02_Preproc_fMRI_{ses}',f'pb03.{sbj}.e02.volreg.{ATLAS_NAME}_000.netcc')
    netcc = load_netcc(netcc_path)
    fcs.loc['.'.join([sbj,ses]),:,:] = netcc.values

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 436/436 [00:07<00:00, 60.38it/s]


In [26]:
 netcc

,001,002,003,004,005,006,007,008,009,010,...,220,221,222,223,224,225,226,227,228,229
001,1.0000,0.4732,0.1939,0.2409,0.3102,0.3205,0.3168,0.4569,0.3239,-0.1439,...,0.3295,0.1695,-0.1284,0.2335,0.1077,0.3670,0.1097,0.3188,0.0664,-0.1104
002,0.4732,1.0000,-0.0709,0.3908,0.7204,0.7468,0.6643,0.7741,0.7744,-0.5958,...,0.7095,-0.2396,-0.5598,0.1267,0.1767,0.3462,-0.0925,0.4371,-0.2009,-0.4752
003,0.1939,-0.0709,1.0000,0.3530,-0.0659,-0.0983,-0.0104,-0.1623,-0.0725,0.1571,...,-0.0658,0.3396,0.1990,0.0088,0.4102,0.1023,0.3579,0.2537,0.6023,0.2485
004,0.2409,0.3908,0.3530,1.0000,0.5361,0.5069,0.5988,0.3661,0.4462,-0.4180,...,0.4894,0.1080,-0.4904,-0.0067,0.3792,0.1934,0.0130,0.2550,0.0462,-0.3394
005,0.3102,0.7204,-0.0659,0.5361,1.0000,0.9307,0.8765,0.7574,0.8607,-0.8354,...,0.8830,-0.2004,-0.8504,-0.0535,0.4068,0.2783,-0.3018,0.4128,-0.3064,-0.6415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0.3670,0.3462,0.1023,0.1934,0.2783,0.2884,0.2451,0.3226,0.2987,-0.1617,...,0.4291,0.2231,-0.1445,0.2056,0.2448,1.0000,0.2079,0.4730,-0.0836,0.1125
226,0.1097,-0.0925,0.3579,0.0130,-0.3018,-0.2521,-0.2364,-0.1708,-0.2393,0.3777,...,-0.2083,0.4009,0.3019,0.1708,0.0674,0.2079,1.0000,0.1561,0.3325,0.4922
227,0.3188,0.4371,0.2537,0.2550,0.4128,0.4142,0.3612,0.3432,0.4258,-0.2258,...,0.4586,0.2232,-0.2565,0.1224,0.6405,0.4730,0.1561,1.0000,0.2060,0.0058
228,0.0664,-0.2009,0.6023,0.0462,-0.3064,-0.2770,-0.2191,-0.1963,-0.2697,0.3729,...,-0.2525,0.4373,0.3868,0.3801,0.2519,-0.0836,0.3325,0.2060,1.0000,0.3939


In [13]:
mot_df = pd.DataFrame(index=np.arange(201),columns=list(dataset_info_df.index))
for sbj,ses in tqdm(list(dataset_info_df.index)):
    mot_path = osp.join(PRCS_DATA_DIR,sbj,f'D02_Preproc_fMRI_{ses}',f'motion_{sbj}_enorm.1D')
    mot_df[(sbj,ses)] = np.loadtxt(mot_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 436/436 [00:00<00:00, 806.57it/s]


In [14]:
scan_select = pn.widgets.Select(name='scan', options=list(fcs.coords['scan'].values))
@pn.depends(scan_select)
def plot_fc(scan):
    aux = fcs.sel(scan=scan).values
    aux = pd.DataFrame(aux,index=roi_info_df.set_index(['ROI_Name','ROI_ID','Hemisphere','Network','RGB']).index, columns=roi_info_df.set_index(['ROI_Name','ROI_ID','Hemisphere','Network','RGB']).index)
    return hvplot_fc(aux,major_label_overrides='regular_grid',cmap='RdBu_r', by='Network', add_labels=False, colorbar_position='left')

In [22]:
@pn.depends(scan_select)
def plot_mot(scan):
    sbj,ses = scan.split('.')
    aux_df = pd.DataFrame(mot_df[(sbj,ses)].values,columns=['Motion [enorm]'])
    aux_df.index.name = 'TR'
    aux_df.name = 'Motion'
    return aux_df.hvplot()

In [23]:
dashboard = pn.Row(scan_select,pn.Column(plot_fc,plot_mot)).show(port=port_tunnel)

Launching server at http://localhost:43169


In [24]:
dashboard.stop()